In [2]:
import pandas as pd
import numpy as np
import pymongo
import requests
import json
import pickle
from collections import defaultdict
from collections import Counter

In [3]:
client = pymongo.MongoClient("mongodb://localhost:37017" )
db=client["lol"]
coll=db["match_data"]

In [4]:
def pull_champions_from_game_data(mongo_object):
    """Takes a dictionary stored in my mongodb.
    Returns ordered lists of allied champions,
    enemy champions, and the winning team id as an int.
    """
    champs=[]
    winning_team=0
    for player in mongo_object["data"]["participants"]:
        champs.append(player["championId"])
    allied_champions=champs[:5]
    enemy_champions=champs[5:]
    if mongo_object["data"]["teams"][0]["win"]=="Fail":
        winning_team=100
    else:
        winning_team=200
    return allied_champions,enemy_champions,winning_team
        

In [12]:
champion_vs=Counter()

In [5]:
def count_matchups(allied_champions,enemy_champions,winning_team):
    """
    Returns counter dict of how many times [key] matchup has occured
    (4,7):3 == champion 4 has *matched against* champion 7 three times
    and
    Returns counter dict of how many times [key] matchup has resulted in a win
    (4,7):3 == champion 4 has *won against* champion 7 three times
    """
    matchups=Counter()
    matchups_wins=Counter()
    for champ in allied_champions:
        for enemychamp in enemy_champions:
            matchups[(champ,enemychamp)]+=1
            if winning_team==100:
                matchups_wins[(champ,enemychamp)]+=1
    for champ in enemy_champions:
        for enemychamp in allied_champions:
            matchups[(champ,enemychamp)]+=1
            if winning_team==200:
                matchups_wins[(champ,enemychamp)]+=1
    return matchups, matchups_wins

In [85]:
# def count_matchups_wins(allied_champions,enemy_champions,winning_team):
#     """Returns counter dict of how many times [key] matchup has resulted in a win
#     (4,7):3 == champion 4 has *won against* champion 7 three times
#     """
#     matchups=Counter()
#     for champ in allied_champions:
#         for enemychamp in enemy_champions:
#             if winning_team==100:
#                 matchups[(champ,enemychamp)]+=1
#     for champ in enemy_champions:
#         for enemychamp in allied_champions:
#             if winning_team==200:
#                 matchups[(champ,enemychamp)]+=1
#     return matchups
    

In [6]:
def total_and_wins_for_matchup(iterable_games_data):
    """
    Calls and aggregates results of count_matchups() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    total_vs=Counter()
    wins_vs=Counter()
    for game in iterable_games_data:
        matchups, matchups_wins=count_matchups(*pull_champions_from_game_data(game))
        total_vs+=matchups
        wins_vs+=matchups_wins
    return total_vs,wins_vs

In [87]:
total, wins = total_and_wins_for_matchup(coll.find({"done":"true"}).limit(20))

In [88]:
sum(total.values()),sum(wins.values())

(1000, 500)

In [7]:
def count_synergies(allied_champions,enemy_champions,winning_team):
    """
    Returns counter dict of how many times [key] synergy has occured
    (4,7):3 == champion 4 has *matched with* champion 7 three times
    and
    Returns counter dict of how many times [key] synergy has resulted in a win
    (4,7):3 == champion 4 has *won with* champion 7 three times
    """
    synergy=Counter()
    synergy_wins=Counter()
    for i,champ in enumerate(allied_champions):
        for i2,champ2 in enumerate(allied_champions):
            if i==i2:
                continue
            synergy[(champ,champ2)]+=1
            if winning_team==100:
                synergy_wins[(champ,champ2)]+=1
    for i,champ in enumerate(enemy_champions):
        for i2,champ2 in enumerate(enemy_champions):
            if i==i2:
                continue
            synergy[(champ,champ2)]+=1
            if winning_team==200:
                synergy_wins[(champ,champ2)]+=1
    return synergy,synergy_wins

In [ ]:
# def count_synergies_wins(allied_champions,enemy_champions,winning_team):
#     """Returns counter dict of how many times [key] synergy has resulted in a win
#     (4,7):3 == champion 4 has *won with* champion 7 three times
#     """
#     synergy=Counter()
#     for i,champ in enumerate(allied_champions):
#         for i2,champ2 in enumerate(allied_champions):
#             if i==i2:
#                 continue
#             if winning_team==100:
#                 synergy[(champ,champ2)]+=1
#     for i,champ in enumerate(enemy_champions):
#         for i2,champ2 in enumerate(enemy_champions):
#             if i==i2:
#                 continue
#             if winning_team==200:
#                 synergy[(champ,champ2)]+=1
#     return synergy
    

In [8]:
def total_and_wins_for_synergy(iterable_games_data):
    """
    Calls and aggregates results of count_synergies() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    total_synergy=Counter()
    wins_synergy=Counter()
    for game in iterable_games_data:
        synergy,synergy_wins=count_synergies(*pull_champions_from_game_data(game))
        total_synergy+=synergy
        wins_synergy+=synergy_wins
    return total_synergy,wins_synergy

In [82]:
total_s,wins_s=total_and_wins_for_synergy(coll.find({"done":"true"}).limit(20))

In [42]:
total, wins = total_and_wins_for_matchup(coll.find({"done":"true"}).limit(10000))

In [43]:
total2=total.copy()
wins2=wins.copy()

In [50]:
len(total2),len(total)

(3548, 18924)

In [49]:
for x in total2.keys():
    if total2[x]<40:
        del total2[x]

In [48]:
sum(total.values())/18000

26

In [13]:
x=total.values()

In [16]:
x.sort(reverse=True)

In [18]:
dd={"a":2,"b":4,"g":7}

In [ ]:
dd.pop

In [51]:
matchup_win_percent={}
for x in total2:
    matchup_win_percent[x]=wins[x]/float(total2[x])

In [52]:
x=matchup_win_percent.values()

In [53]:
x.sort(reverse=True)

In [61]:
def get_value(item):
    return item[1]

In [58]:
xx=matchup_win_percent.items()

In [68]:
xxx=sorted(xx,key=get_value,reverse=True)
xxx

[((412, 59), 0.7446808510638298),
 ((131, 99), 0.7254901960784313),
 ((245, 92), 0.725),
 ((26, 18), 0.725),
 ((58, 40), 0.7209302325581395),
 ((201, 498), 0.7142857142857143),
 ((23, 40), 0.711864406779661),
 ((412, 44), 0.7073170731707317),
 ((236, 114), 0.7021276595744681),
 ((103, 37), 0.7),
 ((64, 119), 0.6981132075471698),
 ((61, 5), 0.6976744186046512),
 ((28, 126), 0.6956521739130435),
 ((81, 63), 0.6949152542372882),
 ((12, 222), 0.6938775510204082),
 ((28, 121), 0.6923076923076923),
 ((21, 43), 0.6904761904761905),
 ((134, 51), 0.6862745098039216),
 ((134, 37), 0.6833333333333333),
 ((17, 37), 0.6825396825396826),
 ((28, 37), 0.6788990825688074),
 ((8, 498), 0.6779661016949152),
 ((236, 110), 0.675),
 ((51, 126), 0.6744186046511628),
 ((15, 29), 0.6727272727272727),
 ((236, 40), 0.6712328767123288),
 ((25, 119), 0.6666666666666666),
 ((202, 57), 0.6666666666666666),
 ((134, 64), 0.6666666666666666),
 ((412, 3), 0.6666666666666666),
 ((119, 103), 0.6666666666666666),
 ((13, 15

In [73]:
for x in range(15):
    print champion_number_to_name(xxx[x][0][0]),champion_number_to_name(xxx[x][0][1]),xxx[x][1]

Thresh Jarvan IV 0.744680851064
Diana Lux 0.725490196078
Ekko Riven 0.725
Zilean Tristana 0.725
Renekton Janna 0.720930232558
Braum Xayah 0.714285714286
Tryndamere Janna 0.71186440678
Thresh Taric 0.707317073171
Lucian Fiora 0.702127659574
Ahri Sona 0.7
Lee Sin Draven 0.698113207547
Orianna Xin Zhao 0.697674418605
Evelynn Jayce 0.695652173913
Ezreal Brand 0.694915254237
Alistar Jinx 0.69387755102


In [70]:
champ_dict=requests.get("https://na1.api.riotgames.com/lol/static-data/v3/champions?locale=en_US&dataById=true&api_key=RGAPI-697d1862-4b7b-4736-9ee8-d8525c958a6f").json()
def champion_number_to_name(champion_number):
    try:
        champ_dict
    except:
        champ_dict = pickle.load(open("champion_dict.p","rb"))
    return champ_dict["data"][str(champion_number)]["name"]

In [94]:
total_s, wins_s = total_and_wins_for_synergy(coll.find({"done":"true"}).limit(6000))

In [95]:
total_s_2=total_s.copy()

In [96]:
for x in total_s_2.keys():
    if total_s_2[x]<20:
        del total_s_2[x]

In [97]:
len(total_s_2),len(total_s)

(3633, 18313)

In [98]:
synergy_win_percent={}
for x in total_s_2:
    synergy_win_percent[x]=wins_s[x]/float(total_s_2[x])

In [99]:
yy=synergy_win_percent.items()

In [100]:
yyy=sorted(yy,key=get_value,reverse=True)
yyy

[((101, 67), 0.8636363636363636),
 ((67, 101), 0.8636363636363636),
 ((236, 14), 0.8571428571428571),
 ((14, 236), 0.8571428571428571),
 ((86, 33), 0.7727272727272727),
 ((33, 86), 0.7727272727272727),
 ((80, 51), 0.7727272727272727),
 ((55, 157), 0.7727272727272727),
 ((157, 55), 0.7727272727272727),
 ((51, 80), 0.7727272727272727),
 ((55, 75), 0.75),
 ((236, 27), 0.75),
 ((27, 236), 0.75),
 ((75, 55), 0.75),
 ((28, 429), 0.7391304347826086),
 ((55, 17), 0.7391304347826086),
 ((17, 55), 0.7391304347826086),
 ((429, 28), 0.7391304347826086),
 ((236, 86), 0.7352941176470589),
 ((86, 236), 0.7352941176470589),
 ((79, 222), 0.7333333333333333),
 ((222, 79), 0.7333333333333333),
 ((67, 85), 0.7307692307692307),
 ((85, 67), 0.7307692307692307),
 ((28, 134), 0.7272727272727273),
 ((110, 126), 0.7272727272727273),
 ((134, 28), 0.7272727272727273),
 ((126, 110), 0.7272727272727273),
 ((161, 81), 0.72),
 ((81, 161), 0.72),
 ((17, 44), 0.7142857142857143),
 ((99, 83), 0.7142857142857143),
 ((44,

In [101]:
for x in range(15):
    print champion_number_to_name(yyy[x][0][0]),champion_number_to_name(yyy[x][0][1]),yyy[x][1]

Xerath Vayne 0.863636363636
Vayne Xerath 0.863636363636
Lucian Sion 0.857142857143
Sion Lucian 0.857142857143
Garen Rammus 0.772727272727
Rammus Garen 0.772727272727
Pantheon Caitlyn 0.772727272727
Katarina Yasuo 0.772727272727
Yasuo Katarina 0.772727272727
Caitlyn Pantheon 0.772727272727
Katarina Nasus 0.75
Lucian Singed 0.75
Singed Lucian 0.75
Nasus Katarina 0.75
Evelynn Kalista 0.739130434783
